In [11]:
Dataset = "FoCus"                                  

LLM_name = "Qwen2-5B-Benchmark"     
# LLM_name = "Qwen2-5B-DPO-AVG"     
# LLM_name = "Qwen2-5B-DPO-LENGTH-PRIOR"   


_COT = ""
# _COT = "-COT"

In [12]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

# ds = pd.read_csv(f'New Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds


Shape: (1000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,Note: Please include all necessary elements in...,0.918397
1,Note: You may need to break down the response ...,0.885235
2,Note: You may need to break down the response ...,0.554154
3,Note: You may need to add additional details i...,0.883863
4,Note: Please include all necessary elements in...,0.895634
...,...,...
995,Note: You may need to add additional details i...,0.787958
996,Note: The response should not contain any pers...,0.894896
997,Note: The response should not contain any pers...,0.725585
998,Note: The response should not contain any pers...,0.912870


In [13]:
print(ds.iloc[167]["gen_response"])

Note: The response should not contain any personal pronouns.

{
  "response": "The Thursday Island Cemetery is a beautiful place to visit. It's a cemetery where many Japanese people have been buried, including sailors and fortune hunters from different countries. The cemetery has over 600 graves, and many of them are Japanese names. The cemetery is also a historical site that showcases important parts of Queensland's history and culture."
}


In [14]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    
    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     390
response_time      0
dtype: int64
(1000, 2)


,gen_response,response_time
0,None,0.918397
1,The Truman Galusha House is a historic house l...,0.885235
2,The Marion Palace Theatre is a movie palace th...,0.554154
3,Technische Universität Darmstadt is a leading ...,0.883863
4,None,0.895634
...,...,...
995,The museum you mentioned is called 'Gaydons Ph...,0.787958
996,None,0.894896
997,Armagh County Museum is a museum located in Ar...,0.725585
998,None,0.912870


In [15]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)